In [1]:
include("common.jl")

using Callbacks
using InvRayTrace: img, img_obs, nointersect, scene
using InvRayTrace: Vec3, Sphere
import InvRayTrace
import RayTrace
import ImageView
using Lens
using Omega
using Omega.Space
using CanSee
using Images
using Plots

using JLD2
const DATADIR = ENV["DATADIR"]
@load "theres.jld2"

In [2]:
"Generate 500 samples using inverse graphics"
generatedata() = InvRayTrace.sampleposterior_noi(500; noi = false, alg = SSMH, gamma = 100.0)

"High resolution render of scene"
highresrender(sc) = RayTrace.rgb.(RayTrace.render(sc; width = 1024, height = 1024))

Main.##370.highresrender

### Actual Causation
In this example we use counterfactuals to determine whether a is the cause of b.
In particular, to determine whether the position of some object a is the cause of us not being able to see object b
We will solve two problems
1. Inverse Graphics, figure out what hte 3d scene is that created a 2d image
2. The actual causation question described above

In principle we could do these simultaneously as one big inference query but to save computation time here we will solve the 1 first and use that data data in the second

### 1. Inverse Graphics

Sample from the posterior of scenes

In [3]:
samples = generatedata()

500-element Array{RayTrace.ListScene{RayTrace.MaterialGeom{GeometryTypes.HyperSphere{3,Float64},RayTrace.Material{GeometryTypes.Vec{3,Float64},Float64,Float64,GeometryTypes.Vec{3,Float64}}}},1}:
 RayTrace.ListScene{RayTrace.MaterialGeom{GeometryTypes.HyperSphere{3,Float64},RayTrace.Material{GeometryTypes.Vec{3,Float64},Float64,Float64,GeometryTypes.Vec{3,Float64}}}}(RayTrace.MaterialGeom{GeometryTypes.HyperSphere{3,Float64},RayTrace.Material{GeometryTypes.Vec{3,Float64},Float64,Float64,GeometryTypes.Vec{3,Float64}}}[MaterialGeom{HyperSphere{3,Float64},Material{Vec{3,Float64},Float64,Float64,Vec{3,Float64}}}(HyperSphere{3,Float64}([0.955483, 0.989129, -17.9624], 2.86956), Material{Vec{3,Float64},Float64,Float64,Vec{3,Float64}}([0.792496, 0.617092, 0.47515], 1.0, 0.0, [0.0, 0.0, 0.0])), MaterialGeom{HyperSphere{3,Float64},Material{Vec{3,Float64},Float64,Float64,Vec{3,Float64}}}(HyperSphere{3,Float64}([-3.9009, -0.664968, -25.5857], 4.85936), Material{Vec{3,Float64},Float64,Float64,Vec{3,

Visualize a single sample from the posterior

In [4]:
ls = samples[end]
ImageView.imshow(highresrender(ls))

Dict{String,Any} with 4 entries:
  "gui"         => Dict{String,Any}("window"=>GtkWindowLeaf(name="", parent, wi…
  "roi"         => Dict{String,Any}("redraw"=>100: "map(clim-mapped image, inpu…
  "annotations" => 53: "input-14" = Dict{UInt64,Any}() Dict{UInt64,Any} 
  "clim"        => 52: "CLim" = CLim{RGB{Float64}}(RGB{Float64}(0.0,0.0,0.0), R…

From a copy of the scene take a single object, Remove object with id gid from scene

In [5]:
ls2 = deepcopy(ls)
gid = 1
ls3 = RayTrace.ListScene([g for (i, g) in enumerate(ls2.geoms) if i!=gid])

k = ls.geoms[gid]

RayTrace.MaterialGeom{GeometryTypes.HyperSphere{3,Float64},RayTrace.Material{GeometryTypes.Vec{3,Float64},Float64,Float64,GeometryTypes.Vec{3,Float64}}}(GeometryTypes.HyperSphere{3,Float64}([-5.37977, -0.0199026, -15.7688], 3.318411805510535), RayTrace.Material{GeometryTypes.Vec{3,Float64},Float64,Float64,GeometryTypes.Vec{3,Float64}}([0.887755, 0.958503, 0.998687], 1.0, 0.0, [0.0, 0.0, 0.0]))

We will add a white ball into the scene inside of k

Define the material colour white

In [6]:
white = RayTrace.Material(Vec3(1.0, 1.0, 0.0), 0.5, 0.0, Vec3(0.0, 0.0, 0.0))

RayTrace.Material{GeometryTypes.Vec{3,Float64},Float64,Float64,GeometryTypes.Vec{3,Float64}}([1.0, 1.0, 0.0], 0.5, 0.0, [0.0, 0.0, 0.0])

Create a new white sphere `mg` at position of `k` but a fraction `frac` of the size

In [7]:
frac = 0.8
mg = RayTrace.MaterialGeom(Sphere(k.center, k.r*frac), white)

RayTrace.MaterialGeom{GeometryTypes.HyperSphere{3,Float64},RayTrace.Material{GeometryTypes.Vec{3,Float64},Float64,Float64,GeometryTypes.Vec{3,Float64}}}(GeometryTypes.HyperSphere{3,Float64}([-5.37977, -0.0199026, -15.7688], 2.654729444408428), RayTrace.Material{GeometryTypes.Vec{3,Float64},Float64,Float64,GeometryTypes.Vec{3,Float64}}([1.0, 1.0, 0.0], 0.5, 0.0, [0.0, 0.0, 0.0]))

Add the new sphere to the scene

In [8]:
push!(ls2.geoms, mg)

"For each element in scene determine if it can be seen"
canseein(scene) = [Bool(CanSee.cansee(scene, scene.geoms[i])) for i = 1:length(scene.geoms)]

canseein(ls2)

9-element Array{Bool,1}:
  true
  true
 false
  true
  true
 false
  true
 false
 false

### 2. Actual Causality

Is fact that object a is where it is,  reason we can't see object b?

In [9]:
objacenter = constant(k.center)
obja = ciid(ω -> RayTrace.MaterialGeom(Sphere(objacenter(ω), k.r), k.material))
objb = mg

sc = ciid(ω -> RayTrace.ListScene([[g for (i, g) in enumerate(ls2.geoms) if i != gid] ; obja(ω)] ))
cantseeb = ciid(ω -> !CanSee.cansee(sc(ω), objb))

1022:getfield(Main.##370, Symbol("##15#16"))()()::Any

Check is fact that in some world ω, is fact that obja where it is cause that we can't see object b

In [10]:
ω = defΩ()()

Omega.Space.LinearΩ{Array{Int64,1},UnitRange{Int64},Array{Any,1}}(Dict{Array{Int64,1},UnitRange{Int64}}(), Any[])

CanSee.iscausebf(ω, objacenter ==ₛ objacenter(ω), cantseeb, [objacenter];
                 sizes = [3],
                 proj = v -> Point(v[1], v[2], v[3]), init = () -> [k.center.data...])#-
*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*